# ファインチューニング
 - 参考 [自然言語処理初心者が「GPT2-japanese」で遊んでみた](https://developer.mamezou-tech.com/blogs/2022/07/08/gpt-2-japanese/)
 - CUDA 12.1以上だとなんかエラー出るっぽい。（推奨は12.0.1）
 - transformers, bitsandbytes, peft, accelerateあたりを最新にしたほうが良さそう。（互換性はあるのだが、よくわからない。）

### 下準備

In [ ]:
!pip install --upgrade pip
!pip install torch
!pip install sentencepiece # [ubuntu にsentencepiece をインストール](https://qiita.com/smoto-shei/items/81f462d624c8fceea8e4)
!pip install transformers
!pip install torchinfo

!pip install -U transformers
!pip install -U bitsandbytes
!pip install -U peft
!pip install -U accelerate

!echo "avionics" | sudo -S apt-get install cmake build-essential pkg-config libgoogle-perftools-dev

!pip install git+https://github.com/huggingface/transformers
!git clone https://github.com/huggingface/transformers
!pip install -r ./transformers/examples/pytorch/language-modeling/requirements.txt

# !pip install accelerate==0.20.3 #0.20.3以上じゃないとファインチューニングでエラーが出る

### 入力文含むプロンプトの設定

In [7]:
# prompt = [
#     {
#         "speaker": "ユーザー",
#         "text": "山地において敵を挟撃するためにどのような戦術が有効ですか？"
#     },
#     # {
#     #     "speaker": "ユーザー",
#     #     "text": "地上最強の生物にふさわしい男は誰ですか？"
#     # },
#     # {
#     #     "speaker": "ユーザー",
#     #     "text": "君たちはどう生きるか"
#     # }
# ]
# prompt = [
#     f"{uttr['speaker']}: {uttr['text']}"
#     for uttr in prompt
# ]
# prompt = "<NL>".join(prompt)
# prompt = (
#     prompt
#     + "<NL>"
#     + "システム: "
# )
# print(prompt)

[{'島嶼における戦術の基本は、'}]


In [ ]:
prompt = [
    {
        "島嶼における戦術の基本は、"
    }
]

print(prompt)

## [rinna/japanese-gpt-1b](https://huggingface.co/rinna/japanese-gpt-1b)

In [8]:
%%time

!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
  --model_name_or_path=rinna/japanese-gpt-1b \
  --train_file=data/tactics_1M.txt \
  --do_train \
  --num_train_epochs=3 \
  --save_steps=10000 \
  --block_size 512 \
  --save_total_limit=3 \
  --per_device_train_batch_size=1 \
  --output_dir=output/ \
  --overwrite_output_dir \
  --use_fast_tokenizer=False

09/12/2023 15:24:25 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
09/12/2023 15:24:25 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_trans

In [ ]:
# %%time
#%%timeとimport timeとの誤差は0.01s程度なので、運用上問題ないと判断し、%%timeの結果で評価する。
%%timeit

# 生成する文章の数
num = 1

import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
tokenizer.do_lower_case = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 学習したモデルを読み込む
model = AutoModelForCausalLM.from_pretrained("output/")
model.to(device)
model.eval()

input_ids = tokenizer.encode(prompt, return_tensors="pt",add_special_tokens=False).to(device)
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=100, # 最長の文章長
        min_length=100, # 最短の文章長
        do_sample=True,
        top_k=500, # 上位{top_k}個の文章を保持
        top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # bad_word_ids=[[tokenizer.unk_token_id]],
        num_return_sequences=num # 生成する文章の数
    )
decoded = tokenizer.batch_decode(output,skip_special_tokens=True)
for i in range(num):
  print(decoded[i])

## [rinna/japanese-gpt2-xsmall](https://huggingface.co/rinna/japanese-gpt2-xsmall)

In [ ]:
%%time

!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
  --model_name_or_path=rinna/japanese-gpt2-xsmall \
  --train_file=data/tactics_1M.txt \
  --do_train \
  --num_train_epochs=3 \
  --save_steps=10000 \
  --block_size 512 \
  --save_total_limit=3 \
  --per_device_train_batch_size=1 \
  --output_dir=output/ \
  --overwrite_output_dir \
  --use_fast_tokenizer=False

In [ ]:
# %%time
#%%timeとimport timeとの誤差は0.01s程度なので、運用上問題ないと判断し、%%timeの結果で評価する。
%%timeit

# 生成する文章の数
num = 1

import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-xsmall")
tokenizer.do_lower_case = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 学習したモデルを読み込む
model = AutoModelForCausalLM.from_pretrained("output/")
model.to(device)
model.eval()

input_ids = tokenizer.encode(prompt, return_tensors="pt",add_special_tokens=False).to(device)
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=100, # 最長の文章長
        min_length=100, # 最短の文章長
        do_sample=True,
        top_k=500, # 上位{top_k}個の文章を保持
        top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # bad_word_ids=[[tokenizer.unk_token_id]],
        num_return_sequences=num # 生成する文章の数
    )
decoded = tokenizer.batch_decode(output,skip_special_tokens=True)
for i in range(num):
  print(decoded[i])

## [rinna/japanese-gpt2-small](https://huggingface.co/rinna/japanese-gpt2-small)

In [ ]:
%%time

!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
  --model_name_or_path=rinna/japanese-gpt2-small \
  --train_file=data/tactics_1M.txt \
  --do_train \
  --num_train_epochs=3 \
  --save_steps=10000 \
  --block_size 512 \
  --save_total_limit=3 \
  --per_device_train_batch_size=1 \
  --output_dir=output/ \
  --overwrite_output_dir \
  --use_fast_tokenizer=False

In [ ]:
# %%time
#%%timeとimport timeとの誤差は0.01s程度なので、運用上問題ないと判断し、%%timeの結果で評価する。
%%timeit

# 生成する文章の数
num = 1

import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
tokenizer.do_lower_case = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 学習したモデルを読み込む
model = AutoModelForCausalLM.from_pretrained("output/")
model.to(device)
model.eval()

input_ids = tokenizer.encode(prompt, return_tensors="pt",add_special_tokens=False).to(device)
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=100, # 最長の文章長
        min_length=100, # 最短の文章長
        do_sample=True,
        top_k=500, # 上位{top_k}個の文章を保持
        top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # bad_word_ids=[[tokenizer.unk_token_id]],
        num_return_sequences=num # 生成する文章の数
    )
decoded = tokenizer.batch_decode(output,skip_special_tokens=True)
for i in range(num):
  print(decoded[i])

## [rinna/japanese-gpt2-medium](https://huggingface.co/rinna/japanese-gpt2-medium)

In [ ]:
%%time

!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
  --model_name_or_path=rinna/japanese-gpt2-medium \
  --train_file=data/tactics_1M.txt \
  --do_train \
  --num_train_epochs=3 \
  --save_steps=10000 \
  --block_size 512 \
  --save_total_limit=3 \
  --per_device_train_batch_size=1 \
  --output_dir=output/ \
  --overwrite_output_dir \
  --use_fast_tokenizer=False

In [ ]:
# %%time
#%%timeとimport timeとの誤差は0.01s程度なので、運用上問題ないと判断し、%%timeの結果で評価する。
%%timeit

# 生成する文章の数
num = 1

import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 学習したモデルを読み込む
model = AutoModelForCausalLM.from_pretrained("output/")
model.to(device)
model.eval()

input_ids = tokenizer.encode(prompt, return_tensors="pt",add_special_tokens=False).to(device)
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=100, # 最長の文章長
        min_length=100, # 最短の文章長
        do_sample=True,
        top_k=500, # 上位{top_k}個の文章を保持
        top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # bad_word_ids=[[tokenizer.unk_token_id]],
        num_return_sequences=num # 生成する文章の数
    )
decoded = tokenizer.batch_decode(output,skip_special_tokens=True)
for i in range(num):
  print(decoded[i])

## [rinna/japanese-gpt-neox-small](https://huggingface.co/rinna/japanese-gpt-neox-small)

In [ ]:
%%time

!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
  --model_name_or_path=rinna/japanese-gpt-neox-small \
  --train_file=data/tactics_1M.txt \
  --do_train \
  --num_train_epochs=3 \
  --save_steps=10000 \
  --block_size 512 \
  --save_total_limit=3 \
  --per_device_train_batch_size=1 \
  --output_dir=output/ \
  --overwrite_output_dir \
  --use_fast_tokenizer=False

In [ ]:
# %%time
#%%timeとimport timeとの誤差は0.01s程度なので、運用上問題ないと判断し、%%timeの結果で評価する。
%%timeit

# 生成する文章の数
num = 1

import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-neox-small")
tokenizer.do_lower_case = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 学習したモデルを読み込む
model = AutoModelForCausalLM.from_pretrained("output/")
model.to(device)
model.eval()

input_ids = tokenizer.encode(prompt, return_tensors="pt",add_special_tokens=False).to(device)
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=100, # 最長の文章長
        min_length=100, # 最短の文章長
        do_sample=True,
        top_k=500, # 上位{top_k}個の文章を保持
        top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # bad_word_ids=[[tokenizer.unk_token_id]],
        num_return_sequences=num # 生成する文章の数
    )
decoded = tokenizer.batch_decode(output,skip_special_tokens=True)
for i in range(num):
  print(decoded[i])

## [rinna/japanese-gpt-neox-3.6b](https://huggingface.co/rinna/japanese-gpt-neox-3.6b)

In [ ]:
%%time

!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
  --model_name_or_path=rinna/japanese-gpt-neox-3.6b \
  --train_file=data/tactics_1M.txt \
  --do_train \
  --num_train_epochs=3 \
  --save_steps=10000 \
  --block_size 512 \
  --save_total_limit=3 \
  --per_device_train_batch_size=1 \
  --output_dir=output/ \
  --overwrite_output_dir \
  --use_fast_tokenizer=False

In [ ]:
# %%time
#%%timeとimport timeとの誤差は0.01s程度なので、運用上問題ないと判断し、%%timeの結果で評価する。
%%timeit

# 生成する文章の数
num = 1

import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-neox-3.6b")
tokenizer.do_lower_case = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 学習したモデルを読み込む
model = AutoModelForCausalLM.from_pretrained("output/")
model.to(device)
model.eval()

input_ids = tokenizer.encode(prompt, return_tensors="pt",add_special_tokens=False).to(device)
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=100, # 最長の文章長
        min_length=100, # 最短の文章長
        do_sample=True,
        top_k=500, # 上位{top_k}個の文章を保持
        top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # bad_word_ids=[[tokenizer.unk_token_id]],
        num_return_sequences=num # 生成する文章の数
    )
decoded = tokenizer.batch_decode(output,skip_special_tokens=True)
for i in range(num):
  print(decoded[i])

## [rinna/bilingual-gpt-neox-4b](https://huggingface.co/rinna/bilingual-gpt-neox-4b)

In [ ]:
%%time

!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
  --model_name_or_path=rinna/bilingual-gpt-neox-4b \
  --train_file=data/tactics_1M.txt \
  --do_train \
  --num_train_epochs=3 \
  --save_steps=10000 \
  --block_size 512 \
  --save_total_limit=3 \
  --per_device_train_batch_size=1 \
  --output_dir=output/ \
  --overwrite_output_dir \
  --use_fast_tokenizer=False

In [ ]:
# %%time
#%%timeとimport timeとの誤差は0.01s程度なので、運用上問題ないと判断し、%%timeの結果で評価する。
%%timeit

# 生成する文章の数
num = 1

import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b")
tokenizer.do_lower_case = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 学習したモデルを読み込む
model = AutoModelForCausalLM.from_pretrained("output/")
model.to(device)
model.eval()

input_ids = tokenizer.encode(prompt, return_tensors="pt",add_special_tokens=False).to(device)
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=100, # 最長の文章長
        min_length=100, # 最短の文章長
        do_sample=True,
        top_k=500, # 上位{top_k}個の文章を保持
        top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # bad_word_ids=[[tokenizer.unk_token_id]],
        num_return_sequences=num # 生成する文章の数
    )
decoded = tokenizer.batch_decode(output,skip_special_tokens=True)
for i in range(num):
  print(decoded[i])

## [rinna/bilingual-gpt-neox-4b-8k](https://huggingface.co/rinna/bilingual-gpt-neox-4b-8k)

In [ ]:
%%time

!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
  --model_name_or_path=rinna/bilingual-gpt-neox-4b-8k \
  --train_file=data/tactics_1M.txt \
  --do_train \
  --num_train_epochs=3 \
  --save_steps=10000 \
  --block_size 512 \
  --save_total_limit=3 \
  --per_device_train_batch_size=1 \
  --output_dir=output/ \
  --overwrite_output_dir \
  --use_fast_tokenizer=False

In [ ]:
# %%time
#%%timeとimport timeとの誤差は0.01s程度なので、運用上問題ないと判断し、%%timeの結果で評価する。
%%timeit

# 生成する文章の数
num = 1

import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b-8k")
tokenizer.do_lower_case = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 学習したモデルを読み込む
model = AutoModelForCausalLM.from_pretrained("output/")
model.to(device)
model.eval()

input_ids = tokenizer.encode(prompt, return_tensors="pt",add_special_tokens=False).to(device)
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=100, # 最長の文章長
        min_length=100, # 最短の文章長
        do_sample=True,
        top_k=500, # 上位{top_k}個の文章を保持
        top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # bad_word_ids=[[tokenizer.unk_token_id]],
        num_return_sequences=num # 生成する文章の数
    )
decoded = tokenizer.batch_decode(output,skip_special_tokens=True)
for i in range(num):
  print(decoded[i])